In [6]:
import cv2
import os
import numpy as np

input_folder = r"C:\Users\abely\OneDrive\Desktop\tomography\tom_snimki"
output_folder = r"C:\Users\abely\OneDrive\Desktop\tomography\tom_snimki_id"

# Функция для нахождения ROI на изображении с помощью неоднородного серого круга
def find_roi(image):
    # Преобразуем изображение в оттенки серого
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Применяем адаптивную пороговую обработку для получения двоичного изображения
    _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # Находим контуры на двоичном изображении
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Если найдены контуры, выбираем наибольший
    if len(contours) > 0:
        roi = max(contours, key=cv2.contourArea)
        
        # Возвращаем координаты ограничивающего круга (x, y, radius) ROI
        (x, y), radius = cv2.minEnclosingCircle(roi)
        return (int(x), int(y), int(radius))
    
    return None

# Получаем список файлов с изображениями в папке
image_files = os.listdir(input_folder)

# Обрабатываем каждый файл
for file in image_files:
    # Считываем изображение
    image = cv2.imread(os.path.join(input_folder, file))
    
    # Находим ROI
    roi = find_roi(image)
    
    if roi is not None:
        # Получаем координаты ROI
        x, y, radius = roi
        
        # Создаем новое изображение с черным фоном
        new_image = np.zeros_like(image)
        
        # Вычисляем среднюю яркость внутри ROI
        roi_mean_brightness = np.mean(image[y-int(radius/2):y+int(radius/2), x-int(radius/2):x+int(radius/2)])
        
        # Вычисляем цвет для серого региона
        gray_color = int(roi_mean_brightness)
        
        # Рисуем неоднородный серый круг на черном фоне
        cv2.circle(new_image, (x, y), int(radius), (gray_color, gray_color, gray_color), -1)
        
        # Сохраняем новое изображение с тем же именем в другую папку
        cv2.imwrite(os.path.join(output_folder, file), new_image)
        
        
        
        # Рисуем контур и маркер центра фигуры
        cv2.circle(image, (x, y), 5, (0, 255, 0), -1)  # Маркер центра фигуры выделен зеленым цветом
        cv2.circle(image, (x, y), int(radius), (0, 255, 0), 2)  # Рисуем окружность на изображении
        
        # Чтобы сохранить исходное изображение до обрезки
        cropped_image = image.copy()

        mask = np.zeros_like(image)
        cv2.circle(mask, (x, y), int(radius), (255, 255, 255), -1)
        
        # Применение битовой маски к изображению
        cropped_image = cv2.bitwise_and(cropped_image, mask)

        cv2.imshow("Cropped Image", cropped_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        #cv2.imshow("Original Image", image)
        #cv2.waitKey(0)
        
cv2.destroyAllWindows()